In [115]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import math


In [121]:

%matplotlib inline
pd.options.display.max_rows = 200
pd.options.display.max_columns = 200

In [3]:
df_train = pd.read_csv("data\\train.csv")
df_test = pd.read_csv("data\\test.csv")
df_mck = pd.read_excel("data\\MCK.xlsx")
df_columns = pd.read_excel("data\\column_comparison.xlsx")
df = pd.concat([df_train, df_test], keys=['train', 'test'])

In [4]:
df_train.shape , df_test.shape

((3865, 80), (890, 79))

In [5]:
# df_train.head(1)

In [6]:
# df_test.head(1)

In [7]:
df_train[["galactic year"]].max() , df_test["galactic year"].min()

(galactic year    1015056
 dtype: int64,
 1007012)

In [8]:
"""
@params:
    df=dataframe
    targetVariable = Target variable
    numberOfColumns = number of values against targetVariable should be compared with (top k variables with highest correlation)
    largestCorrelation : true if values with largest corrleation is wanted and false otherwise
"""
def getTopCorrelationColumnsNames(df, targetVariable, numberOfColumns, largestCorrelation):
    corrmat = df.corr()
    if largestCorrelation==True:
        return corrmat.nlargest(numberOfColumns, targetVariable)[targetVariable].index
    else:
        return corrmat.nsmallest(numberOfColumns, targetVariable)[targetVariable].index
    

In [9]:
topColumns = getTopCorrelationColumnsNames(df_train,"y",31,True).tolist()
len(topColumns)
df_train_short = df_train[topColumns]


In [10]:
# df_train_short.shape,df_train_short.isnull().any()

In [11]:
df_train.shape , df_mck.shape , df_columns.shape , df_train_short.shape

((3865, 80), (5277, 128), (75, 2), (3865, 31))

##### 1. combine train test
##### 2. rename trainTest
##### 3. find std dev of each feature for each country / galaxy on trainTest and mck data
##### 4. apply knn

In [12]:
trainColumns = df_columns["TRAIN"].tolist()
trainMckColumns = df_columns["MCK"].tolist()

columnDictionary = {}
for train_col, trainMck_col in zip(trainColumns,trainMckColumns) :
    columnDictionary[train_col] = trainMck_col
    

In [13]:
df_renamed = df.rename(columns=columnDictionary,inplace=False)

In [110]:
# confirm taking mean
df_CombinedGrouped = df_renamed.groupby("Country").mean()
df_mck = df_mck.groupby("Country").mean()
df_CombinedGrouped_cols = [ (col_name+"_zscore") for col_name in df_CombinedGrouped.columns.tolist() ]


df_renamed_mean = df_renamed.mean()   #mean of  whole dataset for each feature
df_renamed_std = df_renamed.std()    #std of  whole dataset for each feature

df_CombinedGrouped.shape, df_mck.shape

((181, 79), (196, 206))

In [111]:
#function to add columns 
def addColumn(df,columnName,columnValue=0):
    df[columnName] = columnValue
    return df

In [112]:
# adding zscore column to the dataset and each will have a value of 0
for cols in df_CombinedGrouped_cols:
    df_CombinedGrouped = addColumn(df_CombinedGrouped,cols,0)
    df_mck = addColumn(df_mck,cols,0)

In [118]:
for index, row in df_CombinedGrouped.iterrows():
    #   col_name is name of the column and its value is that column's mean
    for col_name in  df_renamed_mean.index.tolist() :
        
        if not math.isnan(row[col_name]):
            df_CombinedGrouped.at[index,(col_name+"_zscore")] = ( row[col_name] - df_renamed_mean[col_name] )  / df_renamed_std[col_name]
        
        

In [122]:
df_CombinedGrouped.head(1)

,year,life expectancy index,Life expectancy at birth,Gross national income (GNI) per capita (2011 PPP$),Income Index,Expected years of schooling (years),Mean years of schooling (years),Human Development Index (HDI),Education Index,"Intergalactic Development Index (IDI), Rank",Population using at least basic drinking-water services (%),Population using at least basic sanitation services (%),Gross capital formation (% of GDP),"Population, total (millions)","Population, urban (%)","Mortality rate, under-five (per 1,000 live births)","Mortality rate, infant (per 1,000 live births)",Old age dependency ratio (old age (65 and older) per 100 creatures (ages 15-64)),"Population, ages 15–64 (millions)","Population, ages 65 and older (millions)","Life expectancy at birth, male (galactic years)","Life expectancy at birth, female (galactic years)","Population, under age 5 (millions)",Young age (0-14) dependency ratio (per 100 people ages 15-64),"Adolescent birth rate (births per 1,000 women ages 15-19)",Total unemployment rate (female to male ratio),Vulnerable employment (% of total employment),"Unemployment, total (% of labour force)",Employment in agriculture (% of total employment),Labour force participation rate (% ages 15 and older),"Labour force participation rate (% ages 15 and older), female",Employment in services (% of total employment),"Labour force participation rate (% ages 15 and older), male",Employment to population ratio (% ages 15 and older),Forest area (% of total land area),"Share of employment in nonagriculture, female (% of total employment in nonagriculture)",Youth unemployment rate (female to male ratio),"Unemployment, youth (% ages 15–24)","Mortality rate, female adult (per 1,000 people)","Mortality rate, male adult (per 1,000 people)","Infants lacking immunization, measles (% of one-year-olds)","Infants lacking immunization, DTP (% of one-year-olds)",Gross domestic product (GDP) per capita (2011 PPP $),"Gross domestic product (GDP), total (2011 PPP $ billions)","Foreign direct investment, net inflows (% of GDP)",Exports and imports (% of GDP),Share of seats in parliament (% held by women),Natural resource depletion (% of GNI),"Mean years of schooling, female (years)","Mean years of schooling, male (years)","Expected years of schooling, female (years)","Expected years of schooling, male (years)","Maternal mortality ratio (deaths per 100,000 live births)",Renewable energy consumption (% of total final energy consumption),"Estimated gross national income per capita, male (2011 PPP$)","Estimated gross national income per capita, female (2011 PPP$)",Rural population with access to electricity (%),Domestic credit provided by financial sector (% of GDP),"Population with at least some secondary education, female (% ages 25 and older)","Population with at least some secondary education, male (% ages 25 and older)",Gross fixed capital formation (% of GDP),"Remittances, inflows (% of GDP)",Population with at least some secondary education (% ages 25 and older),International inbound tourists (thousands),"Gross enrolment ratio, primary (% of primary school-age population)","Tuberculosis incidence (per 100,000 people)",Mobile phone subscriptions (per 100 people),"Internet users, total (% of population)",Current health expenditure (% of GDP),"Human Development Index (HDI), female","Human Development Index (HDI), male",Gender Development Index (GDI),"Intergalactic Development Index (IDI), female, Rank","Intergalactic Development Index (IDI), male, Rank",Adjusted net savings (% of GNI),"HIV prevalence, adult (% ages 15-49), total",Private capital flows (% of GDP),Gender Inequality Index (GII),y,year_zscore,life expectancy index_zscore,Life expectancy at birth _zscore,Gross national income (GNI) per capita (2011 PPP$) _zscore,Income Index_zscore,Expected years of schooling (years)_zscore,Mean years of schooling (years) _zscore,Human Development Index (HDI)_zscore,Education Index_zscore,"Intergalactic Development Index (IDI), Rank_z

In [123]:
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# scaler.fit(X_train)

# X_train = scaler.transform(X_train)
# X_test = scaler.transform(X_test)

In [124]:
# from sklearn.neighbors import KNeighborsClassifier
# classifier = KNeighborsClassifier(n_neighbors=5)
# classifier.fit(X_train, y_train)

In [125]:
# y_pred = classifier.predict(X_test)


In [126]:
# from sklearn.metrics import classification_report, confusion_matrix
# print(confusion_matrix(y_test, y_pred))
# print(classification_report(y_test, y_pred))